In [0]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.datasets import load_iris

TensorFlow 2.x selected.


In [0]:
x, y = load_iris(return_X_y=True)
x.shape, y.shape, set(y) # 클래스는 3개이다

((150, 4), (150,), {0, 1, 2})

In [0]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2020)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((120, 4), (30, 4), (120,), (30,))

In [0]:
class MyModel(keras.Model):

  def __init__(self):
    super(MyModel, self).__init__() # 상속받은 클래스 생성자 호출
    self.opt = tf.optimizers.Adam(learning_rate=0.0001) # stochatic Gradient Descent 확률적 경사 하강
    self.dense = keras.layers.Dense(units=3, activation=keras.activations.softmax) # units = 뉴런수

  def call(self, x):
    h = self.dense(x)
    return h
  
  def get_loss(self, y, h):
    # 학습할 때 nan이 발생하는 경우 clip으로 최소값, 최대값을 정해준다.
    h = tf.clip_by_value(h, 1e-8, 1-1e-8) # h가 0이나 1이 되지 않도록 하는 장치
    cross_entropy = -(y*tf.math.log(h) + (1-y)*tf.math.log(1-h))
    loss = tf.reduce_mean(cross_entropy)
    return loss

  def get_accuracy(self, y, h):
    # h : (m, 3) , y : (m)
    predict = tf.argmax(h, -1)
    self.acc = tf.reduce_mean(tf.cast(tf.equal(y, predict), tf.float32)) # cast 하면 True -> 1, False -> 0

  def train(self, x, y, epoch=1):
    # x : (m, 4), y : (m)

    y_hot = tf.one_hot(y, 3) # (m, 3)
    for i in range(epoch):
      with tf.GradientTape() as tape: # 경사기록장치
        h = self.call(x)
        loss = self.get_loss(y_hot, h)

      grads = tape.gradient(loss, self.trainable_variables)
      self.opt.apply_gradients(zip(grads, self.trainable_variables))
      self.get_accuracy(y, h)
      print('%d/%d loss : %.3f    acc :%.3f'%(i+1, epoch, loss, self.acc))
    
model = MyModel()

In [0]:
model.train(x_train, y_train, 100)

텐서플로 홈페이지에 있는 expert 버전으로 해보기

In [0]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2020, stratify=y)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((120, 4), (30, 4), (120,), (30,))

In [0]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.d1 = Dense(3, activation='softmax', input_shape=(None, 4))

  def call(self, x):
    # x(none, 4)
    h = self.d1(x) # (none, 3)
    h = tf.clip_by_value(h, 1e-8, 1-1e-8)
    return h

model = MyModel()  

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables) # loss에 대한 파라미터들의 도함수
  # list of (gradients, variables) pairs
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)
  
  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(16)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(10000).batch(16)

In [0]:
# EPOCHS = 100

# for epoch in range(EPOCHS):
#   for images, labels in train_ds:
#     train_step(images, labels)

#   for test_images, test_labels in test_ds:
#     test_step(test_images, test_labels)

#   template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
#   print (template.format(epoch+1,
#                          train_loss.result(),
#                          train_accuracy.result()*100,
#                          test_loss.result(),
#                          test_accuracy.result()*100))

In [0]:
EPOCHS = 100

for epoch in range(EPOCHS):
  
  train_step(x_train, y_train)
  test_step(x_test, y_test)
  template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
  print(template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

에포크: 1, 손실: 0.4735454022884369, 정확도: 89.16666412353516, 테스트 손실: 0.44424811005592346, 테스트 정확도: 90.0
에포크: 2, 손실: 0.47337615489959717, 정확도: 89.16666412353516, 테스트 손실: 0.44407305121421814, 테스트 정확도: 90.0
에포크: 3, 손실: 0.4732073247432709, 정확도: 89.16666412353516, 테스트 손실: 0.4438983201980591, 테스트 정확도: 90.0
에포크: 4, 손실: 0.47303879261016846, 정확도: 89.16666412353516, 테스트 손실: 0.44372397661209106, 테스트 정확도: 90.0
에포크: 5, 손실: 0.4728706479072571, 정확도: 89.16666412353516, 테스트 손실: 0.4435499310493469, 테스트 정확도: 90.0
에포크: 6, 손실: 0.47270283102989197, 정확도: 89.16666412353516, 테스트 손실: 0.4433762729167938, 테스트 정확도: 90.0
에포크: 7, 손실: 0.4725353717803955, 정확도: 89.16666412353516, 테스트 손실: 0.443202942609787, 테스트 정확도: 90.0
에포크: 8, 손실: 0.4723682701587677, 정확도: 89.16666412353516, 테스트 손실: 0.4430299699306488, 테스트 정확도: 90.0
에포크: 9, 손실: 0.47220149636268616, 정확도: 89.16666412353516, 테스트 손실: 0.4428573548793793, 테스트 정확도: 90.0
에포크: 10, 손실: 0.47203510999679565, 정확도: 89.16666412353516, 테스트 손실: 0.442685067653656, 테스트 정확도: 90.0
에포크: 11, 손실: 

# digits 데이터 불러와서 cnn 모델 직접 만들어보기

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.datasets import load_digits

In [0]:
x, y = load_digits(return_X_y=True)
x.shape, y.shape, set(y)

((1797, 64), (1797,), {0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

In [0]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2020, stratify=y)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1257, 64), (540, 64), (1257,), (540,))

In [0]:
class DigitModel(keras.Model):
  def __init__(self):
    super(DigitModel, self).__init__() # 상속받은 클래스 생성자 호출
    self.conv1 = keras.layers.Conv2D(32, 3, padding='same', input_shape=(8,8,1))
    self.maxp1 = keras.layers.MaxPool2D(padding='same')
    self.conv2 = keras.layers.Conv2D(16, 3, padding='same')
    self.maxp2 = keras.layers.MaxPool2D(padding='same')
    self.flat = keras.layers.Flatten()
    self.dense = keras.layers.Dense(units=10, activation=keras.activations.softmax) # units = 뉴런수

  def call(self, x):
    x = tf.reshape(x, [-1, 8, 8, 1])
    x = self.conv1(x)
    x = self.maxp1(x)
    x = self.conv2(x)
    x = self.maxp2(x)
    x = self.flat(x)
    h = self.dense(x)
    # h = tf.clip_by_value(h, 1e-8, 1-1e-8)
    return h
model = DigitModel()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables) # loss에 대한 파라미터들의 도함수
  # list of (gradients, variables) pairs
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)
  
  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [0]:
# no mini batch
EPOCHS = 1000

for epoch in range(EPOCHS):
  
  train_step(x_train, y_train)
  test_step(x_test, y_test)

  template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

에포크: 1, 손실: 6.357515335083008, 정확도: 12.569610595703125, 테스트 손실: 6.189169406890869, 테스트 정확도: 13.88888931274414
에포크: 2, 손실: 6.272481918334961, 정확도: 13.007159233093262, 테스트 손실: 6.1093597412109375, 테스트 정확도: 14.166666984558105
에포크: 3, 손실: 6.190745830535889, 정확도: 13.550782203674316, 테스트 손실: 6.032680034637451, 테스트 정확도: 14.629629135131836
에포크: 4, 손실: 6.11216926574707, 정확도: 14.120923042297363, 테스트 손실: 5.958951950073242, 테스트 정확도: 14.907407760620117
에포크: 5, 손실: 6.036545753479004, 정확도: 14.558473587036133, 테스트 손실: 5

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(16)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(10000).batch(16)

In [0]:
# mini batch
EPOCHS = 120

for epoch in range(EPOCHS):

  for x, y in train_ds:
    train_step(x, y)
  
  for x, y in test_ds:
    test_step(x, y)

  template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

에포크: 1, 손실: 2.9919817447662354, 정확도: 16.388225555419922, 테스트 손실: 2.1527631282806396, 테스트 정확도: 25.370370864868164
에포크: 2, 손실: 2.4396979808807373, 정확도: 27.247413635253906, 테스트 손실: 1.9406343698501587, 테스트 정확도: 36.296295166015625
에포크: 3, 손실: 2.133037805557251, 정확도: 37.045875549316406, 테스트 손실: 1.764265775680542, 테스트 정확도: 44.938270568847656
에포크: 4, 손실: 1.9093728065490723, 정확도: 44.94828796386719, 테스트 손실: 1.6177682876586914, 테스트 정확도: 51.25
에포크: 5, 손실: 1.7320829629898071, 정확도: 51.23309326171875, 테스트 손실: 1.493891

In [0]:
class DigitModel(keras.Model):
  def __init__(self):
    super(DigitModel, self).__init__() # 상속받은 클래스 생성자 호출
    self.opt = tf.optimizers.Adam(learning_rate=0.005) # stochatic Gradient Descent 확률적 경사 하강
    self.conv1 = keras.layers.Conv2D(32, 3, padding='same', input_shape=(8,8,1))
    self.maxp1 = keras.layers.MaxPool2D(padding='same')
    self.conv2 = keras.layers.Conv2D(16, 3, padding='same')
    self.maxp2 = keras.layers.MaxPool2D(padding='same')
    self.flat = keras.layers.Flatten()
    self.dense = keras.layers.Dense(units=10, activation=keras.activations.softmax) # units = 뉴런수

  def call(self, x):
    x = tf.reshape(x, [-1, 8, 8, 1])
    x = self.conv1(x)
    x = self.maxp1(x)
    x = self.conv2(x)
    x = self.maxp2(x)
    x = self.flat(x)
    h = self.dense(x)
    return h
  
  def get_loss(self, y, h):
    # 학습할 때 nan이 발생하는 경우 clip으로 최소값, 최대값을 정해준다.
    h = tf.clip_by_value(h, 1e-8, 1-1e-8) # h가 0이나 1이 되지 않도록 하는 장치
    cross_entropy = -(y*tf.math.log(h) + (1-y)*tf.math.log(1-h))
    loss = tf.reduce_mean(cross_entropy)
    return loss

  def get_accuracy(self, y, h):
    # h : (m, 3) , y : (m)
    predict = tf.argmax(h, -1)
    self.acc = tf.reduce_mean(tf.cast(tf.equal(y, predict), tf.float32)) # cast 하면 True -> 1, False -> 0

  def train(self, x, y, epoch=1):
    # x : (m, 4), y : (m)

    y_hot = tf.one_hot(y, 10) # (m, 10)
    for i in range(epoch):
      with tf.GradientTape() as tape: # 경사기록장치
        h = self.call(x)
        loss = self.get_loss(y_hot, h)

      grads = tape.gradient(loss, self.trainable_variables)
      self.opt.apply_gradients(zip(grads, self.trainable_variables))
      self.get_accuracy(y, h)
      print('%d/%d loss : %.3f    acc :%.3f'%(i+1, epoch, loss, self.acc))
    
model = DigitModel()

In [0]:
model.train(x_train, y_train, 75)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1/75 loss : 0.792    acc :0.102
2/75 loss : 0.629    acc :0.088
3/75 loss : 0.586    acc :0.130
4/75 loss : 0.509    acc :0.169
5/75 loss : 0.449    acc :0.140
6/75 loss : 0.437    acc :0.187
7/75 loss : 0.409    acc :0.307
8/75 loss : 0.379    acc :0.345
9/75 loss : 0.340    acc :0.422
10/75 loss : 0.304    acc :0.496
11/75 loss : 0.283    acc :0.545
12/75 loss : 0.268    acc :0.546
13/75 loss : 0.247    acc :0.575
14/75 loss : 0.224    acc :0.629
15/75 loss : 0.205    acc :0.632
16/75 loss : 0.185    